# Final Project

# Comparing Neural Networks for NHL Performance Forecasting

**Group ID: 44**

**Members:<br>
Trevor Lee - 100751771**

The goal here is to use the NHL Player Statistics dataset from Kaggle to compare two different models and see which is more accurate for predicting future performance in the NHL. The dataset contains stats from roughly 1979-1980 to 2019-2020. The selected models will be used to project performance for select players of different ages and positions to project their performance in the 2020-2021 season and then the predictions will be compared to the actual results to see which were more accurate. Only select players will be used because the 2020-2021 season stats are not included in the dataset and I will have to compare them manually. The goal will be to assess and compare the viability of the chosen models to predict player performance.

We will be using the NHL Player Statistics dataset by Benzik on Kaggle (https://www.kaggle.com/datasets/alexbenzik/nhl-players-statistics)

## Preparing the Data

The first thing we need to do is prepare the data so that it is suitable for training.

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Load the data set as a dataframe. The values are serparated by semi-colons instead of commas, so I need to redefine the serparator.

In [2]:
df = pd.read_csv('NHL_Players_Statistics.csv', sep=';')

df.head()

,Name,Date_of_birth,SEASON_year,SEASON,TEAM,Games_Played,Goals,Assists,Points,PlusMinus_Ratings,...,Saves,Save_Percentage,Shutouts,Position,Height,Weight,Body_mass_index,Place_of_birth,Age,Experience
0,Aaron Downey,1974-08-27,2000,'99-'00,BOS,1,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,Right_wing,185,98,28.6,"Shelburne, Ontario",26,1
1,Aaron Downey,1974-08-27,2001,'00-'01,CHI,3,0.0,0.0,0.0,-1.0,...,NaN,NaN,NaN,Right_wing,185,98,28.6,"Shelburne, Ontario",27,2
2,Aaron Downey,1974-08-27,2002,'01-'02,CHI,36,1.0,0.0,1.0,-2.0,...,NaN,NaN,NaN,Right_wing,185,98,28.6,"Shelburne, Ontario",28,3
3,Aaron Downey,1974-08-27,2003,'02-'03,DAL,43,1.0,1.0,2.0,1.0,...,NaN,NaN,NaN,Right_wing,185,98,28.6,"Shelburne, Ontario",29,4
4,Aaron Downey,1974-08-27,2004,'03-'04,DAL,37,1.0,1.0,2.0,2.0,...,NaN,NaN,NaN,Right_wing,185,98,28.6,"Shelburne, Ontario",30,5


Now let's take a look at properties of the dataset.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27319 entries, 0 to 27318
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Name                   27319 non-null  object 
 1   Date_of_birth          27319 non-null  object 
 2   SEASON_year            27319 non-null  int64  
 3   SEASON                 27319 non-null  object 
 4   TEAM                   27319 non-null  object 
 5   Games_Played           27319 non-null  int64  
 6   Goals                  24802 non-null  float64
 7   Assists                24802 non-null  float64
 8   Points                 24802 non-null  float64
 9   PlusMinus_Ratings      24802 non-null  float64
 10  Penalty_Minutes        24802 non-null  float64
 11  Shots_on_Goal          24802 non-null  float64
 12  Shooting_Percentage    24802 non-null  float64
 13  PowerPlay_Goals        24802 non-null  float64
 14  PowerPlay_Assists      24802 non-null  float64
 15  Sh

The first thing we need to trim from the dataset is players with the position of `'Goaltender'`. These players have their stats recorded differently than skaters do and can cause issues.

In [4]:
df = df[df['Position'] != 'Goaltender'].copy()

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24802 entries, 0 to 27318
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Name                   24802 non-null  object 
 1   Date_of_birth          24802 non-null  object 
 2   SEASON_year            24802 non-null  int64  
 3   SEASON                 24802 non-null  object 
 4   TEAM                   24802 non-null  object 
 5   Games_Played           24802 non-null  int64  
 6   Goals                  24802 non-null  float64
 7   Assists                24802 non-null  float64
 8   Points                 24802 non-null  float64
 9   PlusMinus_Ratings      24802 non-null  float64
 10  Penalty_Minutes        24802 non-null  float64
 11  Shots_on_Goal          24802 non-null  float64
 12  Shooting_Percentage    24802 non-null  float64
 13  PowerPlay_Goals        24802 non-null  float64
 14  PowerPlay_Assists      24802 non-null  float64
 15  Short_G

The reduced number of entries and the goalie stats all having a count of 0 confirms that the correct players were removed.

Next we want to only keep the relevant columns or ones that could be relevant rather than keep all 40 columns.

In [5]:
cols = [
    'Name',
    'Date_of_birth',    # needed to differentiate players with the same name playing in the same year
    'SEASON_year',
    'Games_Played',
    'Goals',
    'Assists',
    'Points',
    'Shots_on_Goal',
    'Shooting_Percentage',
    'PowerPlay_Goals',
    'PowerPlay_Assists',
    'Short_Goals',
    'Short_Assists',
    'Position',
    'Height',
    'Weight',
    'Body_mass_index',
    'Age',
    'Experience'
]

df = df[cols].copy()

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24802 entries, 0 to 27318
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Name                 24802 non-null  object 
 1   Date_of_birth        24802 non-null  object 
 2   SEASON_year          24802 non-null  int64  
 3   Games_Played         24802 non-null  int64  
 4   Goals                24802 non-null  float64
 5   Assists              24802 non-null  float64
 6   Points               24802 non-null  float64
 7   Shots_on_Goal        24802 non-null  float64
 8   Shooting_Percentage  24802 non-null  float64
 9   PowerPlay_Goals      24802 non-null  float64
 10  PowerPlay_Assists    24802 non-null  float64
 11  Short_Goals          24802 non-null  float64
 12  Short_Assists        24802 non-null  float64
 13  Position             24802 non-null  object 
 14  Height               24802 non-null  int64  
 15  Weight               24802 non-null  int6

In [6]:
df.head()

,Name,Date_of_birth,SEASON_year,Games_Played,Goals,Assists,Points,Shots_on_Goal,Shooting_Percentage,PowerPlay_Goals,PowerPlay_Assists,Short_Goals,Short_Assists,Position,Height,Weight,Body_mass_index,Age,Experience
0,Aaron Downey,1974-08-27,2000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Right_wing,185,98,28.6,26,1
1,Aaron Downey,1974-08-27,2001,3,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,Right_wing,185,98,28.6,27,2
2,Aaron Downey,1974-08-27,2002,36,1.0,0.0,1.0,10.0,10.0,0.0,0.0,0.0,0.0,Right_wing,185,98,28.6,28,3
3,Aaron Downey,1974-08-27,2003,43,1.0,1.0,2.0,14.0,7.1,0.0,0.0,0.0,0.0,Right_wing,185,98,28.6,29,4
4,Aaron Downey,1974-08-27,2004,37,1.0,1.0,2.0,11.0,9.1,0.0,0.0,0.0,0.0,Right_wing,185,98,28.6,30,5


Next we need to handle instances where the player swaps teams for whatever reason (i.e. trade, waivers) within a given season. This will be done by aggregating their stats.

In [7]:
# These are the columns used to identify a player
id_cols = [
    'Name',
    'Date_of_birth',
    'SEASON_year'
]

# These are the stats to be summed within a season if there are multiple entries
# for a player
sum_cols = [
    'Games_Played',
    'Goals',
    'Assists',
    'Points',
    'Shots_on_Goal',
    'PowerPlay_Goals',
    'PowerPlay_Assists',
    'Short_Goals',
    'Short_Assists',
]

# These are the other columns with data we want to keep and should theoretically
# be the same within the given season
other_cols =[
    'Position',
    'Height',
    'Weight',
    'Body_mass_index',
    'Age',
    'Experience',
]

# Here we define how the data will be aggregated
agg_dict = {col: 'sum' for col in sum_cols}
agg_dict.update({col: 'first' for col in other_cols})

df = df.groupby(id_cols, as_index=False).agg(agg_dict)

# We also need to recompute shooting percentage
# This will also be marginally more accurate since the dataset only has one
# decimal point
df['Shooting_Percentage'] = np.where(
    df['Shots_on_Goal'] > 0,                    # with zero div protection
    df['Goals'] / df['Shots_on_Goal'] * 100,
    0.0
)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22465 entries, 0 to 22464
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Name                 22465 non-null  object 
 1   Date_of_birth        22465 non-null  object 
 2   SEASON_year          22465 non-null  int64  
 3   Games_Played         22465 non-null  int64  
 4   Goals                22465 non-null  float64
 5   Assists              22465 non-null  float64
 6   Points               22465 non-null  float64
 7   Shots_on_Goal        22465 non-null  float64
 8   PowerPlay_Goals      22465 non-null  float64
 9   PowerPlay_Assists    22465 non-null  float64
 10  Short_Goals          22465 non-null  float64
 11  Short_Assists        22465 non-null  float64
 12  Position             22465 non-null  object 
 13  Height               22465 non-null  int64  
 14  Weight               22465 non-null  int64  
 15  Body_mass_index      22465 non-null 

In [8]:
df.head()

,Name,Date_of_birth,SEASON_year,Games_Played,Goals,Assists,Points,Shots_on_Goal,PowerPlay_Goals,PowerPlay_Assists,Short_Goals,Short_Assists,Position,Height,Weight,Body_mass_index,Age,Experience,Shooting_Percentage
0,Aaron Downey,1974-08-27,2000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Right_wing,185,98,28.6,26,1,0.000000
1,Aaron Downey,1974-08-27,2001,3,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,Right_wing,185,98,28.6,27,2,0.000000
2,Aaron Downey,1974-08-27,2002,36,1.0,0.0,1.0,10.0,0.0,0.0,0.0,0.0,Right_wing,185,98,28.6,28,3,10.000000
3,Aaron Downey,1974-08-27,2003,43,1.0,1.0,2.0,14.0,0.0,0.0,0.0,0.0,Right_wing,185,98,28.6,29,4,7.142857
4,Aaron Downey,1974-08-27,2004,37,1.0,1.0,2.0,11.0,0.0,0.0,0.0,0.0,Right_wing,185,98,28.6,30,5,9.090909


Now we need to create "per game" versions for our major stats. These will help our model retain its accuracy in cases where a player had a injury-riddled season.

In [9]:
df['G_per_gp'] = df['Goals'] / df['Games_Played']
df['A_per_gp'] = df['Assists'] / df['Games_Played']
df['P_per_gp'] = df['Points'] / df['Games_Played']
df['SOG_per_gp'] = df['Shots_on_Goal'] / df['Games_Played']

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22465 entries, 0 to 22464
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Name                 22465 non-null  object 
 1   Date_of_birth        22465 non-null  object 
 2   SEASON_year          22465 non-null  int64  
 3   Games_Played         22465 non-null  int64  
 4   Goals                22465 non-null  float64
 5   Assists              22465 non-null  float64
 6   Points               22465 non-null  float64
 7   Shots_on_Goal        22465 non-null  float64
 8   PowerPlay_Goals      22465 non-null  float64
 9   PowerPlay_Assists    22465 non-null  float64
 10  Short_Goals          22465 non-null  float64
 11  Short_Assists        22465 non-null  float64
 12  Position             22465 non-null  object 
 13  Height               22465 non-null  int64  
 14  Weight               22465 non-null  int64  
 15  Body_mass_index      22465 non-null 

In [10]:
df.head()

,Name,Date_of_birth,SEASON_year,Games_Played,Goals,Assists,Points,Shots_on_Goal,PowerPlay_Goals,PowerPlay_Assists,...,Height,Weight,Body_mass_index,Age,Experience,Shooting_Percentage,G_per_gp,A_per_gp,P_per_gp,SOG_per_gp
0,Aaron Downey,1974-08-27,2000,1,0.0,0.0,0.0,0.0,0.0,0.0,...,185,98,28.6,26,1,0.000000,0.000000,0.000000,0.000000,0.000000
1,Aaron Downey,1974-08-27,2001,3,0.0,0.0,0.0,2.0,0.0,0.0,...,185,98,28.6,27,2,0.000000,0.000000,0.000000,0.000000,0.666667
2,Aaron Downey,1974-08-27,2002,36,1.0,0.0,1.0,10.0,0.0,0.0,...,185,98,28.6,28,3,10.000000,0.027778,0.000000,0.027778,0.277778
3,Aaron Downey,1974-08-27,2003,43,1.0,1.0,2.0,14.0,0.0,0.0,...,185,98,28.6,29,4,7.142857,0.023256,0.023256,0.046512,0.325581
4,Aaron Downey,1974-08-27,2004,37,1.0,1.0,2.0,11.0,0.0,0.0,...,185,98,28.6,30,5,9.090909,0.027027,0.027027,0.054054,0.297297
